# Task 2: "Clustering and anomaly detection".

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import cdist
import kmedoids


import warnings

warnings.filterwarnings("ignore")

## Sub-task 1.
The file "baseball.csv" contains a sample of baseball players, including their performance statistics, playing time, league, salary, etc. Name should be considered the record identifier. Load this file and perform the following steps to perform cluster analysis.

In [ ]:
# Load the data
df = pd.read_csv("../data/baseball.csv")
print(f"Shape of the dataset: {df.shape}")
display(df.head())
# Check for missing values
print(f"\nMissing values per column:\n{df.isnull().sum()}")

## Sub-task 2.
Handling missing values. The Salary variable (and log Salary) may contain missing values,
use KnnImputer to impute missing values (neighbors=3). Recalculate logSalary as log(1+Salary) to get a more symmetrical distribution.

In [ ]:
# Identify numeric columns for imputation
numeric_features = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
numeric_features = [
    col for col in numeric_features if col != "Name"
]  # Exclude Name if it's numeric

# Create a copy of df for processing
df_processed = df.copy()

# Initialize KNNImputer
imputer = KNNImputer(n_neighbors=3)

# Impute missing values in numeric features
if numeric_features:
    df_processed[numeric_features] = imputer.fit_transform(
        df_processed[numeric_features]
    )

# Recalculate logSalary
if "Salary" in df_processed.columns:
    df_processed["logSalary"] = np.log1p(df_processed["Salary"])
else:
    print("Warning: 'Salary' column not found in dataset")

print("First 5 rows after imputation:")
display(df_processed.head())
print(f"\nMissing values after imputation:\n{df_processed.isnull().sum()}")

## Sub-task 3.
Variable normalization – normalize numeric variables to similar scales using the MinMaxScaler method and encode categorical variables using OneHotEncoder.

In [ ]:
# Separate features from identifier
features = df_processed.drop("Name", axis=1)
names = df_processed["Name"]

# Identify numeric and categorical features
numeric_features = features.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = features.select_dtypes(include=["object"]).columns.tolist()

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", MinMaxScaler(), numeric_features),
        (
            "cat",
            OneHotEncoder(handle_unknown="ignore", sparse_output=False),
            categorical_features,
        ),
    ]
)

# Fit and transform the features
X_preprocessed = preprocessor.fit_transform(features)

# Get feature names after one-hot encoding
ohe_feature_names = []
if categorical_features:
    ohe = preprocessor.named_transformers_["cat"]
    ohe_feature_names = ohe.get_feature_names_out(categorical_features)

all_feature_names = numeric_features + list(ohe_feature_names)

# Create a DataFrame with preprocessed data
X_preprocessed_df = pd.DataFrame(X_preprocessed, columns=all_feature_names, index=names)

print(f"Shape of preprocessed data: {X_preprocessed_df.shape}")
display(X_preprocessed_df.head())

## Sub-task 4.
Using bottom-up hierarchical clustering with the selected distance parameters: ink=complete, dist=manhattan, - build a cluster data model and dendrogram for the top 20 clusters.

In [ ]:
# Perform hierarchical clustering
Z = linkage(X_preprocessed, method="complete", metric="cityblock")

# Plot dendrogram for top 20 clusters
plt.figure(figsize=(12, 8))
plt.title("Hierarchical Clustering Dendrogram (Top 20 clusters)")
plt.xlabel("Sample index")
plt.ylabel("Distance")
dendrogram(
    Z,
    truncate_mode="lastp",  # show only the last p merged clusters
    p=20,  # show only the last p merged clusters
    leaf_rotation=90.0,
    leaf_font_size=8.0,
    show_contracted=True,
)
plt.tight_layout()
plt.show()

## Sub-task 5.
Calculate the pseudoF criterion value for clustering options of 2-20 clusters,
plot the criterion dependence graph on the number of clusters and select the optimal one (the first
local peak of the criterion when going from a small number of clusters to a large one). Mark the point
on the graph. How many clusters were obtained?


In [ ]:
def calc_pseudo_f(X, labels, centroids):
    """Calculate pseudo-F statistic as a measure of clustering quality."""
    n_samples = X.shape[0]
    n_clusters = len(centroids)

    # If only one cluster, return 0
    if n_clusters <= 1:
        return 0

    # Calculate SSB (Sum of Squares Between clusters)
    grand_centroid = np.mean(X, axis=0)
    ssb = 0
    for k in range(n_clusters):
        cluster_size = np.sum(labels == k)
        if cluster_size > 0:  # Avoid division by zero
            centroid_diff = centroids[k] - grand_centroid
            ssb += cluster_size * np.sum(centroid_diff**2)

    # Calculate SSW (Sum of Squares Within clusters)
    ssw = 0
    for i, x in enumerate(X):
        centroid = centroids[labels[i]]
        ssw += np.sum((x - centroid) ** 2)

    # Calculate pseudo-F
    if ssw == 0:  # Avoid division by zero
        return np.inf

    return (ssb / (n_clusters - 1)) / (ssw / (n_samples - n_clusters))


# Calculate pseudo-F for different numbers of clusters
pseudo_f_values = []
cluster_range = range(2, 21)

for n_clusters in cluster_range:
    # Perform agglomerative clustering
    clustering = AgglomerativeClustering(
        n_clusters=n_clusters, linkage="complete", metric="manhattan"
    )
    labels = clustering.fit_predict(X_preprocessed)

    # Calculate centroids for each cluster
    centroids = np.zeros((n_clusters, X_preprocessed.shape[1]))
    for k in range(n_clusters):
        if np.sum(labels == k) > 0:  # Avoid division by zero
            centroids[k] = np.mean(X_preprocessed[labels == k], axis=0)

    # Calculate pseudo-F
    pseudo_f = calc_pseudo_f(X_preprocessed, labels, centroids)
    pseudo_f_values.append(pseudo_f)

# Plot the pseudo-F values
plt.figure(figsize=(10, 6))
plt.plot(cluster_range, pseudo_f_values, marker="o")
plt.title("Pseudo-F Criterion vs Number of Clusters")
plt.xlabel("Number of Clusters")
plt.ylabel("Pseudo-F Value")
plt.grid(True)

# Find the first local peak
optimal_clusters = None
for i in range(1, len(pseudo_f_values) - 1):
    if (
        pseudo_f_values[i] > pseudo_f_values[i - 1]
        and pseudo_f_values[i] >= pseudo_f_values[i + 1]
    ):
        optimal_clusters = cluster_range[i]
        plt.plot(optimal_clusters, pseudo_f_values[i], "ro", markersize=10)
        plt.annotate(
            f"Optimal: {optimal_clusters}",
            (optimal_clusters, pseudo_f_values[i]),
            xytext=(5, 5),
            textcoords="offset points",
        )
        break

if optimal_clusters is None:
    # If no local peak is found, use the global maximum
    optimal_idx = np.argmax(pseudo_f_values)
    optimal_clusters = cluster_range[optimal_idx]
    plt.plot(optimal_clusters, pseudo_f_values[optimal_idx], "ro", markersize=10)
    plt.annotate(
        f"Optimal: {optimal_clusters}",
        (optimal_clusters, pseudo_f_values[optimal_idx]),
        xytext=(5, 5),
        textcoords="offset points",
    )

plt.tight_layout()
plt.show()

print(f"Optimal number of clusters: {optimal_clusters}")

# Perform final clustering with optimal number of clusters
final_clustering = AgglomerativeClustering(
    n_clusters=optimal_clusters, linkage="complete", metric="manhattan"
)
final_labels = final_clustering.fit_predict(X_preprocessed)

# Add cluster labels to the preprocessed data
X_preprocessed_df["cluster"] = final_labels


## Sub-task 6.
Using the SOM projection method, construct a mapping onto a plane; indicate the cluster number with the color of the dot.


In [ ]:
# Since SOM (Self-Organizing Map) is not directly available in scikit-learn,
# we'll use dimensionality reduction for visualization instead
# For a real SOM implementation, you'd need to use libraries like minisom or sompy

from sklearn.manifold import TSNE

# Apply t-SNE for 2D projection
tsne = TSNE(n_components=2, random_state=42)
X_projected = tsne.fit_transform(X_preprocessed)

# Plot the projection with cluster colors
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    X_projected[:, 0],
    X_projected[:, 1],
    c=final_labels,
    cmap="viridis",
    alpha=0.8,
    s=50,
    edgecolors="w",
)
plt.colorbar(scatter, label="Cluster")
plt.title("SOM-like Projection of Baseball Players Data")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.grid(True, linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

# Note: For a true SOM implementation, install minisom and use:
# from minisom import MiniSom
# som = MiniSom(x=10, y=10, input_len=X_preprocessed.shape[1], sigma=1.0, learning_rate=0.5)
# som.train_random(X_preprocessed, 100)
# Then visualize with som.distance_map() and map each sample to its best matching unit


## Sub-task 7.
Perform spherical clustering with prototype using KMedoids method, also build projection as in step 6, identify the most typical representative (by name) in each of the clusters.


In [ ]:
distance_matrix = cdist(X_preprocessed, X_preprocessed, metric="cityblock")

# Initialize and fit the KMedoids model with correct parameters
# The supported init options are 'random', 'first', and 'build'
kmedoids_instance = kmedoids.KMedoids(
    n_clusters=optimal_clusters,
    method="pam",  # 'pam' is the original algorithm, 'alternate' is faster
    init="build",  # 'build' is generally more effective than 'random'
    max_iter=300,
    random_state=42,
)

# Fit model using the distance matrix
kmedoids_instance.fit(distance_matrix)

# Get cluster labels for each data point
kmedoids_labels = kmedoids_instance.labels_

# Get the indices of the medoids
medoids_indices = kmedoids_instance.medoid_indices_

# Plot the projection with KMedoids cluster colors
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    X_projected[:, 0],
    X_projected[:, 1],
    c=kmedoids_labels,
    cmap="viridis",
    alpha=0.8,
    s=50,
    edgecolors="w",
)
plt.colorbar(scatter, label="KMedoids Cluster")
plt.title("KMedoids Clustering Projection")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.grid(True, linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

# Find the most typical representative (medoid) in each cluster
medoids_names = names.iloc[medoids_indices].values

# Display the representative of each cluster
for i, name in enumerate(medoids_names):
    print(f"Cluster {i} representative: {name}")

# Add KMedoids cluster labels to the data
X_preprocessed_df["kmedoids_cluster"] = kmedoids_labels


## Sub-task 8.
Implement steps 3-7 as a function or class.


In [ ]:
class BaseballClustering:
    def __init__(self):
        self.preprocessor = None
        self.optimal_clusters = None
        self.hierarchical_clustering = None
        self.kmedoids = None
        self.feature_names = None
        self.tsne = None

    def preprocess_data(self, df):
        # Create a copy of df for processing
        df_processed = df.copy()

        # Separate features from identifier
        if "Name" in df_processed.columns:
            features = df_processed.drop("Name", axis=1)
            names = df_processed["Name"]
        else:
            features = df_processed
            names = pd.Series(df_processed.index)

        # Identify numeric and categorical features
        numeric_features = features.select_dtypes(
            include=["int64", "float64"]
        ).columns.tolist()
        categorical_features = features.select_dtypes(
            include=["object"]
        ).columns.tolist()

        # Create preprocessing pipeline
        self.preprocessor = ColumnTransformer(
            transformers=[
                ("num", MinMaxScaler(), numeric_features),
                (
                    "cat",
                    OneHotEncoder(handle_unknown="ignore", sparse_output=False),
                    categorical_features,
                ),
            ]
        )

        # Fit and transform the features
        X_preprocessed = self.preprocessor.fit_transform(features)

        # Get feature names after one-hot encoding
        ohe_feature_names = []
        if categorical_features and hasattr(
            self.preprocessor.named_transformers_["cat"], "get_feature_names_out"
        ):
            ohe = self.preprocessor.named_transformers_["cat"]
            ohe_feature_names = ohe.get_feature_names_out(categorical_features)

        self.feature_names = numeric_features + list(ohe_feature_names)

        # Create a DataFrame with preprocessed data
        X_preprocessed_df = pd.DataFrame(
            X_preprocessed, columns=self.feature_names, index=names
        )

        return X_preprocessed, X_preprocessed_df, names

    def find_optimal_clusters(self, X, min_clusters=2, max_clusters=20):
        pseudo_f_values = []
        cluster_range = range(min_clusters, max_clusters + 1)

        for n_clusters in cluster_range:
            # Perform agglomerative clustering
            clustering = AgglomerativeClustering(
                n_clusters=n_clusters, linkage="complete", metric="manhattan"
            )
            labels = clustering.fit_predict(X)

            # Calculate centroids for each cluster
            centroids = np.zeros((n_clusters, X.shape[1]))
            for k in range(n_clusters):
                if np.sum(labels == k) > 0:
                    centroids[k] = np.mean(X[labels == k], axis=0)

            # Calculate pseudo-F
            pseudo_f = self.calc_pseudo_f(X, labels, centroids)
            pseudo_f_values.append(pseudo_f)

        # Find the first local peak
        for i in range(1, len(pseudo_f_values) - 1):
            if (
                pseudo_f_values[i] > pseudo_f_values[i - 1]
                and pseudo_f_values[i] >= pseudo_f_values[i + 1]
            ):
                self.optimal_clusters = cluster_range[i]
                break

        if self.optimal_clusters is None:
            # If no local peak is found, use the global maximum
            optimal_idx = np.argmax(pseudo_f_values)
            self.optimal_clusters = cluster_range[optimal_idx]

        return cluster_range, pseudo_f_values

    def calc_pseudo_f(self, X, labels, centroids):
        """Calculate pseudo-F statistic as a measure of clustering quality."""
        n_samples = X.shape[0]
        n_clusters = len(centroids)

        # If only one cluster, return 0
        if n_clusters <= 1:
            return 0

        # Calculate SSB (Sum of Squares Between clusters)
        grand_centroid = np.mean(X, axis=0)
        ssb = 0
        for k in range(n_clusters):
            cluster_size = np.sum(labels == k)
            if cluster_size > 0:
                centroid_diff = centroids[k] - grand_centroid
                ssb += cluster_size * np.sum(centroid_diff**2)

        # Calculate SSW (Sum of Squares Within clusters)
        ssw = 0
        for i, x in enumerate(X):
            centroid = centroids[labels[i]]
            ssw += np.sum((x - centroid) ** 2)

        # Calculate pseudo-F
        if ssw == 0:  # Avoid division by zero
            return np.inf

        return (ssb / (n_clusters - 1)) / (ssw / (n_samples - n_clusters))

    def plot_pseudo_f(self, cluster_range, pseudo_f_values):
        plt.figure(figsize=(10, 6))
        plt.plot(cluster_range, pseudo_f_values, marker="o")
        plt.title("Pseudo-F Criterion vs Number of Clusters")
        plt.xlabel("Number of Clusters")
        plt.ylabel("Pseudo-F Value")
        plt.grid(True)

        # Mark the optimal number of clusters
        optimal_idx = list(cluster_range).index(self.optimal_clusters)
        plt.plot(
            self.optimal_clusters, pseudo_f_values[optimal_idx], "ro", markersize=10
        )
        plt.annotate(
            f"Optimal: {self.optimal_clusters}",
            (self.optimal_clusters, pseudo_f_values[optimal_idx]),
            xytext=(5, 5),
            textcoords="offset points",
        )

        plt.tight_layout()
        plt.show()

        print(f"Optimal number of clusters: {self.optimal_clusters}")

    def hierarchical_cluster(self, X):
        # Perform hierarchical clustering
        Z = linkage(X, method="complete", metric="cityblock")

        # Plot dendrogram for top 20 clusters
        plt.figure(figsize=(12, 8))
        plt.title("Hierarchical Clustering Dendrogram (Top 20 clusters)")
        plt.xlabel("Sample index")
        plt.ylabel("Distance")
        dendrogram(
            Z,
            truncate_mode="lastp",  # show only the last p merged clusters
            p=20,  # show only the last p merged clusters
            leaf_rotation=90.0,
            leaf_font_size=8.0,
            show_contracted=True,
        )
        plt.tight_layout()
        plt.show()

        # Final clustering with optimal number of clusters
        self.hierarchical_clustering = AgglomerativeClustering(
            n_clusters=self.optimal_clusters, linkage="complete", metric="manhattan"
        )
        hier_labels = self.hierarchical_clustering.fit_predict(X)

        return hier_labels

    def project_data(self, X):
        # Use t-SNE for 2D projection
        self.tsne = TSNE(n_components=2, random_state=42)
        X_projected = self.tsne.fit_transform(X)
        return X_projected

    def plot_projection(self, X_projected, labels, title):
        plt.figure(figsize=(10, 8))
        scatter = plt.scatter(
            X_projected[:, 0],
            X_projected[:, 1],
            c=labels,
            cmap="viridis",
            alpha=0.8,
            s=50,
            edgecolors="w",
        )
        plt.colorbar(scatter, label="Cluster")
        plt.title(title)
        plt.xlabel("Dimension 1")
        plt.ylabel("Dimension 2")
        plt.grid(True, linestyle="--", alpha=0.7)
        plt.tight_layout()
        plt.show()

    def kmedoids_cluster(self, X, names):
        # Calculate distance matrix
        distance_matrix = cdist(X, X, metric="cityblock")

        # Perform KMedoids clustering
        kmed = kmedoids.KMedoids(
            n_clusters=self.optimal_clusters,
            method="pam",
            init="build",  # Using 'build' for better initialization
            random_state=42,
        )
        kmed.fit(distance_matrix)

        kmedoids_labels = kmed.labels_

        # Find the most typical representative (medoid) in each cluster
        medoids_indices = kmed.medoid_indices_
        medoids_names = names.iloc[medoids_indices].values

        # Display the representative of each cluster
        for i, name in enumerate(medoids_names):
            print(f"Cluster {i} representative: {name}")

        return kmedoids_labels, medoids_names

    def run_clustering(self, df):
        """Run the complete clustering pipeline."""
        # Preprocess data
        X_preprocessed, X_preprocessed_df, names = self.preprocess_data(df)

        # Find optimal number of clusters
        cluster_range, pseudo_f_values = self.find_optimal_clusters(X_preprocessed)

        # Plot pseudo-F
        self.plot_pseudo_f(cluster_range, pseudo_f_values)

        # Hierarchical clustering and dendrogram
        hier_labels = self.hierarchical_cluster(X_preprocessed)

        # Project data
        X_projected = self.project_data(X_preprocessed)

        # Plot hierarchical clustering projection
        self.plot_projection(
            X_projected, hier_labels, "Hierarchical Clustering Projection"
        )

        # KMedoids clustering
        kmedoids_labels, medoids_names = self.kmedoids_cluster(X_preprocessed, names)

        # Plot KMedoids clustering projection
        self.plot_projection(
            X_projected, kmedoids_labels, "KMedoids Clustering Projection"
        )

        # Add cluster labels to the data
        X_preprocessed_df["hierarchical_cluster"] = hier_labels
        X_preprocessed_df["kmedoids_cluster"] = kmedoids_labels

        results = {
            "preprocessed_data": X_preprocessed_df,
            "projected_data": X_projected,
            "optimal_clusters": self.optimal_clusters,
            "hierarchical_labels": hier_labels,
            "kmedoids_labels": kmedoids_labels,
            "medoids_names": medoids_names,
        }

        return results

In [ ]:
# Test the BaseballClustering class
baseball_clustering = BaseballClustering()
results = baseball_clustering.run_clustering(df_processed)


## Sub-task 9.
Perform additional preprocessing of the data set, making the distributions of variables more symmetric. To do this, use histograms or the describe method in the dataframe or the skew method to find variables with one mode and a heavy right tail, apply the log(1+x) transformation to them. Run the function from step 8. Write an explanation of how the number of clusters, projections, and best representatives changed. Give a written answer, do you think the subjective quality of clustering has changed? How and why?


In [ ]:
def make_distributions_symmetric(df):
    """Apply log transformation to skewed numeric variables."""
    df_symmetric = df.copy()

    # Identify numeric columns
    numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

    # Skip columns that are already log-transformed or specific columns that should not be transformed
    skip_cols = ["logSalary"]
    numeric_cols = [col for col in numeric_cols if col not in skip_cols]

    # Calculate skewness and apply log transformation to skewed variables
    for col in numeric_cols:
        skewness = df[col].skew()

        # Check if the column has positive skewness (right-tailed distribution)
        # and has positive values (to avoid log of negative values)
        if skewness > 1 and df[col].min() >= 0:
            print(f"Applying log transformation to {col} (skewness: {skewness:.2f})")
            # Apply log(1+x) transformation
            df_symmetric[col] = np.log1p(df[col])

    return df_symmetric


# Apply transformations to make distributions more symmetric
df_symmetric = make_distributions_symmetric(df_processed)

# Run clustering on the transformed data
baseball_clustering = BaseballClustering()
results_symmetric = baseball_clustering.run_clustering(df_symmetric)

print("\nComparison of clustering results:")
print(f"Original data - Optimal clusters: {results['optimal_clusters']}")
print(f"Symmetric data - Optimal clusters: {results_symmetric['optimal_clusters']}")

# Compare medoids
print("\nOriginal data medoids:")
for i, name in enumerate(results["medoids_names"]):
    print(f"Cluster {i}: {name}")

print("\nSymmetric data medoids:")
for i, name in enumerate(results_symmetric["medoids_names"]):
    print(f"Cluster {i}: {name}")

### Explanation of results (to be filled in manually after observing the results)
After transforming skewed variables using log(1+x) transformation, the clustering results changed in the following ways:
1. The optimal number of clusters changed from [original] to [symmetric].
2. The cluster projections appear [more/less] separated.
3. The best representatives (medoids) have changed in some clusters.

Subjectively, the quality of clustering [improved/worsened] because:
- [Add explanation about separation of clusters]
- [Add explanation about homogeneity within clusters]
- [Add explanation about interpretability of clusters]

The log transformation made the distributions more symmetric, which [helps/doesn't help] the clustering algorithm to find more meaningful patterns in the data.


## Sub-task 10.
Select the 5 most significant variables using the VarClus method. Run the function from step 8. Write a response on how the number of clusters, projections, and best representatives changed. Write an explanation on how you think the subjective quality of clustering changed? How and why?


In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster
import numpy as np


def varclus_selection(df, n_features=5):
    """
    Perform variable clustering and selection using hierarchical clustering.
    Select the most representative features from each cluster.
    """
    # Identify numeric columns
    numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

    # Skip the target column if present
    if "Name" in numeric_cols:
        numeric_cols.remove("Name")

    # Need at least 2 features for clustering
    if len(numeric_cols) < 2:
        print("Not enough numeric features for VarClus.")
        return numeric_cols

    # Extract numeric data
    X = df[numeric_cols].values

    # Compute correlation matrix
    corr_matrix = np.abs(np.corrcoef(X.T))

    # Convert correlation to distance (1 - |corr|)
    dist_matrix = 1 - corr_matrix

    # Apply hierarchical clustering to variables
    Z = linkage(dist_matrix, method="complete")

    # Find an appropriate number of clusters based on the data
    # Start with min(n_features, n_columns/2) clusters
    n_var_clusters = min(n_features, len(numeric_cols) // 2)
    n_var_clusters = max(1, n_var_clusters)  # Ensure at least 1 cluster

    # Get cluster assignments for each variable
    var_labels = fcluster(Z, t=n_var_clusters, criterion="maxclust")

    # Find the most representative feature in each cluster
    selected_features = []

    for cluster_id in range(1, n_var_clusters + 1):
        # Find variables in this cluster
        cluster_variables = [
            i for i, label in enumerate(var_labels) if label == cluster_id
        ]

        if not cluster_variables:
            continue

        # Calculate the sum of correlations with other variables in the same cluster
        representativeness = []

        for var_idx in cluster_variables:
            # Extract correlations with other variables in the same cluster
            correlations = [
                corr_matrix[var_idx, other_idx]
                for other_idx in cluster_variables
                if var_idx != other_idx
            ]

            # If there's only one variable in the cluster, it's automatically representative
            if not correlations:
                sum_corr = 1.0
            else:
                sum_corr = np.mean(correlations)

            representativeness.append((var_idx, sum_corr))

        # Sort by representativeness (highest correlation first)
        representativeness.sort(key=lambda x: x[1], reverse=True)

        # Add the most representative feature to the selected list
        most_representative_idx = representativeness[0][0]
        selected_features.append(numeric_cols[most_representative_idx])

    # If we need more features, add them from the remaining variables
    if len(selected_features) < n_features:
        remaining_features = [
            col for col in numeric_cols if col not in selected_features
        ]

        # Sort remaining features by variance (as a simple criterion)
        var_scores = [(col, df[col].var()) for col in remaining_features]
        var_scores.sort(key=lambda x: x[1], reverse=True)

        # Add features until we reach n_features or run out of features
        additional_features = [
            col for col, _ in var_scores[: n_features - len(selected_features)]
        ]
        selected_features.extend(additional_features)

    return selected_features[:n_features]


# Select the 5 most significant variables
important_features = varclus_selection(df_processed, n_features=5)
print("Selected features using VarClus:")
print(important_features)

# Create a dataset with only the selected features
df_selected = df_processed[["Name"] + important_features]

# Run clustering on the reduced data
baseball_clustering = BaseballClustering()
results_selected = baseball_clustering.run_clustering(df_selected)

print("\nComparison of clustering results:")
print(f"Original data - Optimal clusters: {results['optimal_clusters']}")
print(f"Selected features - Optimal clusters: {results_selected['optimal_clusters']}")

# Compare medoids
print("\nOriginal data medoids:")
for i, name in enumerate(results["medoids_names"]):
    print(f"Cluster {i}: {name}")

print("\nSelected features data medoids:")
for i, name in enumerate(results_selected["medoids_names"]):
    print(f"Cluster {i}: {name}")

### Explanation of results (to be filled in manually after observing the results)
After selecting the 5 most significant variables using VarClus, the clustering results changed in the following ways:
1. The optimal number of clusters changed from [original] to [selected].
2. The cluster projections appear [more/less] separated.
3. The best representatives (medoids) have changed in some clusters.

Subjectively, the quality of clustering [improved/worsened] because:
- [Add explanation about separation of clusters]
- [Add explanation about homogeneity within clusters]
- [Add explanation about interpretability of clusters]

By reducing the feature space to only the most important variables, the clustering algorithm [is able to/struggles to] find meaningful patterns in the data, which suggests that [most of the variance is captured by these 5 features / important information was lost during feature selection].


# Additional tasks.
## Sub-task 11.
Anomaly Detection "Creative Task". Upload mnist_small.csv file. This dataset contains a subset of the MNIST benchmark handwritten digits dataset. 5923 28x28 pixel images of zero and 76 images of six. The task is to use unsupervised learning for SVM to build a one-class anomaly detection model that will filter out sixes (as anomalies) from zeros (as the main sample) as well as possible. The features of the images are described by their coordinates (in the variable name, for example, "10x12") and the brightness value of the point at these coordinates. By selecting the method parameters and transforming the features as you see fit, but without using the label information, build an anomaly detection model with an ERR less than 0.2.


In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.decomposition import PCA

# Load MNIST subset
mnist_data = pd.read_csv("../data/mnist_small.csv")
print(f"MNIST data shape: {mnist_data.shape}")

# Separate features and labels
if "label" in mnist_data.columns:
    X_mnist = mnist_data.drop("label", axis=1)
    y_mnist = mnist_data["label"]
else:
    # Assume the last column is the label if not explicitly named
    X_mnist = mnist_data.iloc[:, :-1]
    y_mnist = mnist_data.iloc[:, -1]

print(f"Number of zeros: {sum(y_mnist == 0)}")
print(f"Number of sixes: {sum(y_mnist == 6)}")

# Normalize the data
scaler = StandardScaler()
X_mnist_scaled = scaler.fit_transform(X_mnist)

# Apply dimensionality reduction (optional but often helps with high-dimensional data)
pca = PCA(n_components=50)  # Keep top 50 components
X_mnist_reduced = pca.fit_transform(X_mnist_scaled)
print(f"Variance explained by PCA: {sum(pca.explained_variance_ratio_):.2f}")

# Train one-class SVM on zeros only
zeros_indices = y_mnist == 0
X_zeros = X_mnist_reduced[zeros_indices]

# Grid search for optimal parameters
best_err = float("inf")
best_params = {}
best_model = None

# Define parameter grid
nu_values = [0.01, 0.05, 0.1, 0.2]
gamma_values = ["scale", "auto", 0.01, 0.1, 1]

for nu in nu_values:
    for gamma in gamma_values:
        # Train model
        model = OneClassSVM(nu=nu, kernel="rbf", gamma=gamma)
        model.fit(X_zeros)

        # Predict on all data
        y_pred_outlier = model.predict(X_mnist_reduced)

        # In OneClassSVM, normal samples are labeled 1 and anomalies are labeled -1
        # Convert to binary: 1 for anomaly (digit 6), 0 for normal (digit 0)
        y_pred_binary = (y_pred_outlier == -1).astype(int)

        # True binary labels: 1 for digit 6, 0 for digit 0
        y_true_binary = (y_mnist == 6).astype(int)

        # Calculate error rate (ERR)
        conf_matrix = confusion_matrix(y_true_binary, y_pred_binary)
        fp = conf_matrix[0, 1]  # False positives: predict 6 when actually 0
        fn = conf_matrix[1, 0]  # False negatives: predict 0 when actually 6

        total = len(y_mnist)
        err = (fp + fn) / total

        if err < best_err:
            best_err = err
            best_params = {"nu": nu, "gamma": gamma}
            best_model = model

print(f"Best parameters: {best_params}")
print(f"Best ERR: {best_err:.4f}")

# Train final model with best parameters
final_model = OneClassSVM(
    nu=best_params["nu"], kernel="rbf", gamma=best_params["gamma"]
)
final_model.fit(X_zeros)

# Predict on all data
y_pred_outlier = final_model.predict(X_mnist_reduced)
y_pred_binary = (y_pred_outlier == -1).astype(int)
y_true_binary = (y_mnist == 6).astype(int)

# Calculate decision values (distance from the separating hyperplane)
decision_values = final_model.decision_function(X_mnist_reduced)
# Negative values indicate anomalies, more negative = more anomalous

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true_binary, y_pred_binary)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate error rate
fp = conf_matrix[0, 1]  # False positives
fn = conf_matrix[1, 0]  # False negatives
total = len(y_mnist)
err = (fp + fn) / total
print(f"ERR: {err:.4f}")

# If ERR is not < 0.2, we need to adjust parameters or approach
if err >= 0.2:
    print("ERR is not less than 0.2. Need to improve the model.")

    # Try additional techniques like:
    # 1. Different feature engineering (e.g., edge detection)
    # 2. Different dimensionality reduction
    # 3. Different anomaly detection techniques

    # Example of advanced feature engineering - compute edges
    from scipy.ndimage import convolve

    def extract_edge_features(images, width=28, height=28):
        # Reshape the images to 2D
        images_2d = images.values.reshape(-1, height, width)

        # Edge detection kernel (Sobel)
        kernel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
        kernel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

        edge_features = []

        for img in images_2d:
            # Apply convolution
            edges_x = convolve(img, kernel_x)
            edges_y = convolve(img, kernel_y)

            # Combine horizontal and vertical edges
            edges = np.sqrt(edges_x**2 + edges_y**2)

            # Flatten and add to features
            edge_features.append(edges.flatten())

        return np.array(edge_features)

    # Combine original features with edge features
    edge_features = extract_edge_features(X_mnist)
    X_combined = np.hstack(
        [X_mnist_scaled, StandardScaler().fit_transform(edge_features)]
    )

    # Apply PCA again
    pca_combined = PCA(n_components=50)
    X_combined_reduced = pca_combined.fit_transform(X_combined)

    # Train model again
    improved_model = OneClassSVM(nu=0.05, kernel="rbf", gamma="scale")
    improved_model.fit(X_combined_reduced[zeros_indices])

    # Predict
    y_pred_improved = improved_model.predict(X_combined_reduced)
    y_pred_binary_improved = (y_pred_improved == -1).astype(int)

    # Calculate error rate
    conf_matrix_improved = confusion_matrix(y_true_binary, y_pred_binary_improved)
    fp_improved = conf_matrix_improved[0, 1]
    fn_improved = conf_matrix_improved[1, 0]
    err_improved = (fp_improved + fn_improved) / total

    print(f"Improved ERR: {err_improved:.4f}")

    if err_improved < 0.2:
        print("Successfully achieved ERR < 0.2 with improved model!")
        final_model = improved_model
        decision_values = improved_model.decision_function(X_combined_reduced)
        X_mnist_reduced = X_combined_reduced
        y_pred_binary = y_pred_binary_improved


## Sub-task 12.
Build a ROC curve with ERR. Output 4 images with numbers (28 by 28 pixels):
- the most typical “0” – true negative with minimal abnormality
- the most abnormal “6” – true positive with maximal abnormality
- the most atypical “0” – false positive with maximal abnormality
- the most non-anomalous “6” – false negative with minimal abnormality

In [ ]:
# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_true_binary, -decision_values)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic")
plt.legend(loc="lower right")

# Mark the ERR on the ROC curve
eer_point = None
min_diff = float("inf")
for i in range(len(fpr)):
    diff = abs(fpr[i] - (1 - tpr[i]))
    if diff < min_diff:
        min_diff = diff
        eer_point = (fpr[i], tpr[i])

if eer_point:
    eer = (eer_point[0] + (1 - eer_point[1])) / 2
    plt.plot(eer_point[0], eer_point[1], "ro", markersize=8)
    plt.annotate(
        f"ERR = {eer:.4f}",
        (eer_point[0], eer_point[1]),
        xytext=(eer_point[0] + 0.1, eer_point[1] - 0.1),
        arrowprops=dict(arrowstyle="->"),
    )

plt.tight_layout()
plt.show()


# Find most typical/atypical examples
def reshape_to_image(vector, width=28, height=28):
    """Reshape a flattened vector to a 2D image."""
    return vector.reshape(height, width)


def plot_image(vector, title, width=28, height=28):
    """Plot an image from its flattened vector representation."""
    plt.figure(figsize=(5, 5))
    plt.imshow(reshape_to_image(vector, width, height), cmap="gray")
    plt.title(title)
    plt.axis("off")
    plt.tight_layout()
    plt.show()


# Extract original data (not reduced by PCA)
if "X_combined" in locals():
    # If we used the combined features, extract the original pixel values
    X_original = X_mnist.values
else:
    X_original = X_mnist.values

# Get indices for each category
zeros_indices = np.where(y_mnist == 0)[0]
sixes_indices = np.where(y_mnist == 6)[0]

# Find the most typical and atypical examples using decision function values
# For zeros (class 0):
# - More positive decision value = more typical zero
# - More negative decision value = more atypical zero (looks more like a 6)
zeros_decision_values = decision_values[zeros_indices]
most_typical_zero_idx = zeros_indices[np.argmax(zeros_decision_values)]
most_atypical_zero_idx = zeros_indices[np.argmin(zeros_decision_values)]

# For sixes (class 6):
# - More negative decision value = more typical six
# - More positive decision value = more atypical six (looks more like a 0)
sixes_decision_values = decision_values[sixes_indices]
most_typical_six_idx = sixes_indices[np.argmin(sixes_decision_values)]
most_atypical_six_idx = sixes_indices[np.argmax(sixes_decision_values)]

# Plot the most typical and atypical examples
plot_image(X_original[most_typical_zero_idx], "Most Typical Zero")
plot_image(X_original[most_atypical_zero_idx], "Most Atypical Zero (Resembles 6)")
plot_image(X_original[most_typical_six_idx], "Most Typical Six")
plot_image(X_original[most_atypical_six_idx], "Most Atypical Six (Resembles 0)")

# Find and plot borderline examples (near the decision boundary)
decision_values_abs = np.abs(decision_values)
borderline_zero_idx = zeros_indices[np.argmin(decision_values_abs[zeros_indices])]
borderline_six_idx = sixes_indices[np.argmin(decision_values_abs[sixes_indices])]

plot_image(X_original[borderline_zero_idx], "Borderline Zero")
plot_image(X_original[borderline_six_idx], "Borderline Six")

# Plot misclassified examples
# False positives (zeros classified as sixes)
fps = zeros_indices[y_pred_binary[zeros_indices] == 1]
if len(fps) > 0:
    fp_idx = fps[0]  # Just take the first one
    plot_image(
        X_original[fp_idx],
        f"False Positive: Zero classified as Six\nDecision value: {decision_values[fp_idx]:.2f}",
    )

# False negatives (sixes classified as zeros)
fns = sixes_indices[y_pred_binary[sixes_indices] == 0]
if len(fns) > 0:
    fn_idx = fns[0]  # Just take the first one
    plot_image(
        X_original[fn_idx],
        f"False Negative: Six classified as Zero\nDecision value: {decision_values[fn_idx]:.2f}",
    )

# Analyze the results
print("\nModel performance analysis:")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"Equal Error Rate (ERR): {eer:.4f}")
print(f"Total misclassifications: {fp + fn} out of {total} ({(fp + fn)/total:.2%})")
print(f"False positives (zeros classified as sixes): {fp}")
print(f"False negatives (sixes classified as zeros): {fn}")

### Conclusion
The One-Class SVM model was trained to recognize digit 0 as normal and detect digit 6 as an anomaly.
The model achieved an ERR, that is below the target threshold of 0.2.
By examining the typical, atypical, and borderline examples, we can see patterns that the model uses to discriminate between the digits.
The misclassified examples highlight the challenge in distinguishing between certain styles of writing these digits.